## Local RAG and LLM Integration

[RAG and LLM Integration](https://apmonitor.com/dde/index.php/Main/RAGLargeLanguageModel) in the [Data-Driven Engineering](http://apmonitor.com/dde) online course.

<img align=left width=500px src='https://apmonitor.com/dde/uploads/Main/RAG_LLM_NLP.png'>

Combining Retrieval-Augmented Generation (RAG) with Large Language Models (LLMs) leads to context-aware systems. RAG optimizes the output of a large language model by referencing an external authoritative knowledge base outside of initial training data sources. These external references generate a response to provide more accurate, contextually relevant, and up-to-date information. In this architecture, the LLM is the reasoning engine while the RAG context provides relevant data. This is different than fine-tuning where the LLM parameters are augmented based on a specific knowledge database.

In [ ]:
import ollama
import pandas as pd
import chromadb

Design function to create the ChromaDB vector store by reading and encoding [train.jsonl from GitHub](https://github.com/BYU-PRISM/GEKKO/blob/master/docs/llm/train.jsonl). More information on this topic is available in [RAG Similarity Search](https://apmonitor.com/dde/index.php/Main/SimilaritySearch).

<img align=left width=300px src='https://apmonitor.com/dde/uploads/Main/similarity_search.png'>

In [ ]:
# Loading and preparing the ChromaDB with data
def setup_chromadb():
    # read Gekko LLM training data
    url='https://raw.githubusercontent.com'
    path='/BYU-PRISM/GEKKO/master/docs/llm/train.jsonl'
    qa = pd.read_json(url+path,lines=True)
    documents = []
    metadatas = []
    ids = []

    for i in range(len(qa)):
        s = f"### Question: {qa['question'].iloc[i]} ### Answer: {qa['answer'].iloc[i]}"
        documents.append(s)
        metadatas.append({'qid': f'qid_{i}'})
        ids.append(str(i))

    cc = chromadb.Client()
    cdb = cc.create_collection(name='gekko')
    cdb.add(documents=documents, metadatas=metadatas, ids=ids)
    return cdb

# Setup ChromaDB
cdb = setup_chromadb()

The train.jsonl file contains hundreds of questions and answers about Gekko. It is used to provide context for the Gekko Support Agent that assists with questions about modeling and optimization in Python. The train.jsonl file is added to lists required to build the vector store with documents with the text, metadatas with a unique ID name, and ids with a unique integer identifier.

Create the `ollama_llm` function to generate a response to a question. This requires the [Ollama Server](https://ollama.com) with the `mistral` model that runs locally. More information on this topic is available in the [Tutorial on LLM with Ollama Python Library](https://apmonitor.com/dde/index.php/Main/LargeLanguageModel).

<img align=left width=300px src='https://apmonitor.com/dde/uploads/Main/ollama_llm.png'>

In [ ]:
# Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

Create the RAG chain function that retrieves related information from the ChromaDB vector database. Feed this information as context with the question.

In [ ]:
# Define the RAG chain
def rag_chain(question, cdb):
    context = cdb.query(query_texts=[question],
                        n_results=5, include=['documents'])
    formatted_context = "\n\n".join(x for x in context['documents'][0])
    formatted_context += "\n\nYou are a professional and technical assistant trained to answer questions about Gekko, which is a high-performance Python package for optimization, simulation, machine learning, data-science, model predictive control, and parameter estimation. In addition, you can also help with answering questions about programming in Python, particularly in relation to the aforementioned topics. Your primary goal is to assist users in finding solutions and gaining knowledge in these areas."
    result = ollama_llm(question, formatted_context)
    return result

Create a prompt for the local RAG LLM and test.

In [ ]:
# Create prompt for Local RAG LLM
question = 'What are you trained to do?'
out = rag_chain(question, cdb)
print(out)

Use the [Gekko AI Assistant](https://apmonitor.com/docs) (cloud solution) to ask the same question.

In [ ]:
from gekko import support
assistant = support.agent()
assistant.ask("What are you trained to do?")